In [1]:
import torch
import torch.nn as nn
import torch.optim as optim6
import matplotlib.pyplot as plt
from torch.autograd import Variable # storing data while learning
from config import CFG ,Args
from baselineUtils import load_datasets,distance_metrics
from utils import ScheduledOptim,visualize_preds,visualize_preds_only
from train import train_attn_mdn 
from test import test_mdn ,inference,inference_realtime
from torch.optim.lr_scheduler import LambdaLR 
from model import Attention_GMM #,Attention_GMM_Encoder,Transformer_MDN
# from torch.utils.data.distributed import  DistributedSampler
# from torch.nn.parallel import DistributedDataParallel as DDP
# from torch.distributed import init_process_group,destroy_process_group
import os

/home/avl1/anaconda3/envs/mdn/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
class person:
    def __init__(self,id,point_box,bbox,dist,height,frame_rate=15): # takes the id and first bounding box of the detected person
        self.id = id
        self.input_size = 8          # input size for point based is 8 frames
        self.index = self.input_size-1 #self.input_size-1
        self.distance = dist
        self.height = height
        self.bounding = bbox
        self.history = np.zeros((self.input_size, 4)) # 6 parameters position(x,y)velocity(x,y),acceleration(x,y) 
        # self.history = [[0]*4]*self.input_size
        # self.his_np = np.zeros((self.input_size, 4))
        self.dt = 4  #1/frame_rate            # change in time
        self.add_box(point_box)
   
    def derivative(self,x, dt=1):
        not_nan_mask = ~np.isnan(x)
        masked_x = x[not_nan_mask]

        if masked_x.shape[-1] < 2:
            return np.zeros_like(x)
        dx = np.full_like(x, np.nan)
    
        dx[not_nan_mask] = np.ediff1d(masked_x, to_begin=(masked_x[1] - masked_x[0]))#/ self.dt
        
        return dx
             

    def add_box(self,box):
        for j in range (self.index,self.input_size-1):
            self.history[j] =  self.history[j+1]
        self.history[self.input_size-1] = box
        if self.index < self.input_size-1:
            self.history[self.input_size-1][2] = self.history[self.input_size-1][0]- self.history[self.input_size-1-1][0]
            self.history[self.input_size-1][3] = self.history[self.input_size-1][1]- self.history[self.input_size-1-1][1]
        #print("HISTORY:", self.history)
        if self.index !=0: 
            self.index-=1       

In [3]:
#################################################################   ZED CAMERA ###############################################################
import pyzed.sl as sl
import cv2
import math
import numpy as np
def infer_real_time(attn_mdn,device,add_features,mean,std):
    # Create a Camera object
    zed = sl.Camera()
    zed_pose = sl.Pose()

    # Create a InitParameters object and set configuration parameters
    init_params = sl.InitParameters()
    init_params.camera_resolution = sl.RESOLUTION.HD1080  # Use HD1080 video mode    
    init_params.coordinate_units = sl.UNIT.METER
    init_params.camera_fps = 15 #cfg_zed['frame_rate']                          # Set fps at 30
    init_params.coordinate_system = sl.COORDINATE_SYSTEM.RIGHT_HANDED_Y_UP
    #camera_position = zed.get_position(zed_pose, sl.REFERENCE_FRAME.CAMERA)

    # Set runtime parameters
    runtime_parameters = sl.RuntimeParameters()

    # Open the camera
    err = zed.open(init_params)
    if err != sl.ERROR_CODE.SUCCESS:
        exit(1)

    # Enable object detection module
    obj_param = sl.ObjectDetectionParameters()
    # Defines if the object detection will track objects across images flow.
    obj_param.enable_tracking = True       # if True, enable positional tracking

    if obj_param.enable_tracking:
        zed.enable_positional_tracking()
        
    zed.enable_object_detection(obj_param)

    camera_info = zed.get_camera_information()
    # Create OpenGL viewer
    # viewer = gl.GLViewer()
    # viewer.init(camera_info.calibration_parameters.left_cam, obj_param.enable_tracking)

    # Configure object detection runtime parameters
    obj_runtime_param = sl.ObjectDetectionRuntimeParameters()
    obj_runtime_param.detection_confidence_threshold = 60
    obj_runtime_param.object_class_filter = [sl.OBJECT_CLASS.PERSON]    # Only detect Persons

    # Create ZED objects filled in the main loop
    objects = sl.Objects()
    image = sl.Mat()
    objects_dic= dict()          # Dictionary holding objects values        

    while zed.grab(runtime_parameters) == sl.ERROR_CODE.SUCCESS:#viewer.is_available():
        # Grab an image, a RuntimeParameters object must be given to grab()
        # if zed.grab(runtime_parameters) == sl.ERROR_CODE.SUCCESS:
        # Retrieve left image
        zed.retrieve_image(image, sl.VIEW.LEFT)
        

        # Retrieve objects
        zed.retrieve_objects(objects, obj_runtime_param)
        batch_dic=dict()    
        input_list =[]       
        if objects.is_new:
            # Count the number of objects detected
            print("{} Object(s) detected".format(len(objects.object_list)))
            if len(objects.object_list):
                obj_detected= False
                for obj in objects.object_list:
                    obj_detected= True
                    distance = math.sqrt(obj.position[0]*obj.position[0] + obj.position[2]*obj.position[2])
                    box = [obj.position[0],obj.position[2],0,0]
                    height = obj.position[1]
                    bounding_box = obj.bounding_box_2d 
                    frame_rate = zed.get_camera_information().camera_fps
                    bbox = np.concatenate((bounding_box[0], bounding_box[2]), axis=0)


                    key = obj.id
                    velocity_is_nan = math.isnan(obj.velocity[0])

                    if key in objects_dic: 
                        objects_dic[key].add_box(box)
                        batch_dic[key] =  objects_dic[key]
                        input_list.append(objects_dic[key].history)
                    elif (not velocity_is_nan):
                        new_person = person(key,box,bbox,distance,height,frame_rate)
                        batch_dic[key] = new_person
                
                objects_dic = batch_dic
        input_dataset = torch.Tensor(input_list)
        test_dl = torch.utils.data.DataLoader(input_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)
       
        batch_preds,candidate_trajs,candidate_weights,best_candiates,src_trajs = inference_realtime(test_dl, attn_mdn,device,add_features = add_features,mixtures=gaussians,enc_seq = 8,dec_seq=12, mode='feed',loss_mode ='mdn',mean=mean,std=std)
        print("batch_preds: ",batch_preds)
        #visualize_preds_only(src_trajs,candidate_trajs,candidate_weights)
        #print("INPUT LIST: ",input_list)
            # for dic_key in objects_dic:
            #     if (objects_dic[dic_key].index < 5):             # wait at least until 4 frames of a person are detected..index decreases starting  from 8
        # Update GL view
        image_ocv = image.get_data() 
        cv2.imshow("Image", image_ocv)
        cv2.waitKey(1)
    image.free(memory_type=sl.MEM.CPU)
    zed.disable_object_detection()
    zed.disable_positional_tracking()
    cv2.destroyAllWindows()
    zed.close()

# Device used and Data loader

In [4]:
device = CFG.device
batch_size = CFG.batch_size
print(f"Using {device} device")
args = Args 

Using cuda:1 device


In [5]:
mean = torch.tensor([-6.5713e-03, -8.1075e-05])
std = torch.tensor([0.3120, 0.1511])

In [6]:
if not Args.real_time:
    train_dataset, val_dataset,test_dataset,mean,std = load_datasets(args)
    train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=CFG.num_workers)
    val_dl = torch.utils.data.DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=CFG.num_workers)
    test_dl = torch.utils.data.DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)

In [7]:
# Train
loss_type = Args.loss_mode
in_features = CFG.in_features
out_features = CFG.out_features
num_heads = CFG.num_heads
num_encoder_layers = CFG.num_encoder_layers
num_decoder_layers =  CFG.num_decoder_layers
embedding_size = CFG.embd_size
max_length = 8
n_hidden = CFG.n_hidden
gaussians = CFG.gaussians
forecast_window = 12
drp = CFG.drop_out
add_features = CFG.add_features

In [8]:
# If you want to train transformer only copy it from commented.py
# Train attention MDN
#attn_mdn = Attention_GMM(device,in_features,out_features,num_heads,num_encoder_layers,num_decoder_layers,embedding_size,n_gaussians=gaussians,n_hidden = n_hidden, dropout=drp).to(device)
attn_mdn = Attention_GMM(in_features,out_features,num_heads,num_encoder_layers,num_decoder_layers,embedding_size,n_gaussians=gaussians,n_hidden = n_hidden, dropout=drp).to(device)
#attn_mdn = Attention_GMM_Encoder(device,in_features,out_features,num_heads,num_encoder_layers,num_decoder_layers,embedding_size,n_gaussians=gaussians,n_hidden = n_hidden, dropout=drp).to(device)
# if torch.cuda.device_count() > 1:
#     print("Using", torch.cuda.device_count(), "GPUs")
#     attn_mdn = DDP(attn_mdn,device_ids=[0,1])
for p in attn_mdn.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
# Define the optimizer
optimizer = ScheduledOptim(
        torch.optim.Adam(attn_mdn.parameters(), betas=(0.9, 0.98), eps=1e-09),
        CFG.lr_mul, CFG.d_model, CFG.n_warmup_steps) #len(train_dl)True
#         print(name, child)

In [9]:
if(Args.mode=='train'):
    loss_train, loss_eval,val_mad,val_fad = train_attn_mdn(train_dl,val_dl,test_dl,attn_mdn,optimizer,add_features,mixtures =gaussians, epochs=CFG.epochs,mean=mean,std=std)
else:
    PATH = Args.model_path
    print("PATH: ",PATH)
    attn_mdn = torch.load(PATH).to(device)

PATH:  saved_models/model_V_zara2_71_H4_E3_032_070.pt


In [10]:
if(Args.mode=='train'):
    fig = plt.figure(1)	#identifies the figure 
    plt.title(" Training Loss Per Epoch", fontsize='16')	#title
    plt.plot(loss_train,color='Blue',label='Training Loss')	#plot the points
    plt.plot(loss_eval,color='Green',label='Evaluation Loss')	#plot the points
    plt.legend(loc="upper right")
    plt.show()

In [11]:
if(Args.mode=='train'):
    fig = plt.figure(2)	#identifies the figure 
    plt.title("Evaluation Error", fontsize='16')	#title
    # plt.plot(test_mad,color='Green', label="ADE Test")	#plot the points
    # plt.plot(test_fad,color='Red', label="FDE Test")	#plot the points
    plt.plot(val_mad,color='Green', label="ADE Validation")	#plot the points
    plt.plot(val_fad,color='Red', label="FDE Validation")	#plot the points
    plt.legend(loc="upper right")
    plt.show()

In [12]:
# Count the number of parameters
num_params = sum(p.numel() for p in attn_mdn.parameters() if p.requires_grad)
print(f"The model has {num_params} trainable parameters.")

The model has 29536248 trainable parameters.


In [13]:
if(Args.mode=='inference'):
    if Args.real_time:
        print("REAL_TIME PREDICTION")
        infer_real_time(attn_mdn,device,add_features,mean,std)
        
    else:
        batch_preds,candidate_trajs,candidate_weights,best_candiates,src_trajs = inference(test_dl, attn_mdn,device,add_features = add_features,mixtures=gaussians,enc_seq = 8,dec_seq=12, mode='feed',loss_mode ='mdn',mean=mean,std=std)
else:
    # Test the model
    batch_preds,batch_gts,avg_mad,avg_fad,candidate_trajs,candidate_weights,best_candiates,src_trajs = test_mdn(test_dl, attn_mdn,device,add_features = add_features,mixtures=gaussians,enc_seq = 8,dec_seq=12, mode='feed',loss_mode ='mdn',mean=mean,std=std)

REAL_TIME PREDICTION
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
1 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
1 Object(s) detected


/tmp/ipykernel_19936/1977077844.py:90: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484772832/work/torch/csrc/utils/tensor_new.cpp:204.)
  input_dataset = torch.Tensor(input_list)


batch_preds:  [array([[[-2.3087041, -3.758687 ],
        [-2.308585 , -3.7586875],
        [-2.3085058, -3.758689 ],
        [-2.3084643, -3.758691 ],
        [-2.3084426, -3.7586956],
        [-2.3084307, -3.7587018],
        [-2.3084245, -3.758708 ],
        [-2.308423 , -3.758713 ],
        [-2.3084261, -3.7587173],
        [-2.3084328, -3.7587206],
        [-2.3084424, -3.7587233],
        [-2.3084536, -3.758725 ]]], dtype=float32)]
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
1 Object(s) detected
batch_preds:  []
1 Object(s) detected
batch_preds:  [array([[[-0.8178681 , -3.3320868 ],
        [-0.69346666, -3.3077092 ],
        [-0.5745094 , -3.2848425 ],
        [-0.36602402, -3.2397149 ],
        [-0.14625156, -3.1948266 ],
        [ 0.08380169, -3.153469  ],
        [ 0.31750363, -3.113671  ],
        [ 0.5525318 , -3.0741994 ],
        [ 0.78839546, -3.0348392 ],
        [ 1.0247998 , -2.9954786 ],
        [ 1.2615647 , -2.9560623 ],
        [ 1.4

Exception ignored in: <function _releaseLock at 0x7f8bcbafe710>
Traceback (most recent call last):
  File "/home/avl1/anaconda3/envs/mdn/lib/python3.10/logging/__init__.py", line 228, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:  []
0 Object(s) detected
batch_preds:

In [ ]:
# # Test the model
# batch_preds,batch_gts,avg_mad,avg_fad,candidate_trajs,candidate_weights,best_candiates,src_trajs = test_mdn(test_dl, attn_mdn,device,add_features = add_features,mixtures=gaussians,enc_seq = 8,dec_seq=12, mode='feed',loss_mode ='mdn',mean=mean,std=std)

shape of candidate trajs (num_batchs, bacth_size, x, 2, 12, 2)  ---> where x is the number of candidate trajectories
shape of ground truth (num_batchs, bacth_size, 12, 2) ---> 

In [ ]:
# Visulaize output via visualize_preds
if Args.visualize and Args.mode=='inference':
    visualize_preds_only(src_trajs,candidate_trajs,candidate_weights)
elif Args.visualize:
    visualize_preds(src_trajs,batch_gts,candidate_trajs,candidate_weights)

In [ ]:
# # Visulaize output via visualize_preds
# if Args.visualize:
#     visualize_preds(src_trajs,batch_gts,candidate_trajs)